In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
txt = "Data\Data 2.txt"
df = pd.read_csv(txt, sep="\t")

In [3]:
df.set_index("type", inplace = True)
df.drop(index = ["type"], inplace = True)
df.reset_index(inplace = True)
# df.head(20)

In [4]:
df.fillna(method = 'ffill', inplace = True)
df["eos"].fillna(1, inplace = True)
df["desc"].fillna(df['name'], inplace = True)
# df.head(10)

C:\Users\LUNA\AppData\Local\Temp\ipykernel_6776\2458029604.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method = 'ffill', inplace = True)
C:\Users\LUNA\AppData\Local\Temp\ipykernel_6776\2458029604.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Area' 'Area' 'Area' ... 'Trees' 'Trees' 'Trees']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df["desc"].fillna(df['name'], inplace = True)


In [5]:
for col in df.columns:
    if df[col].nunique() <= 2:
        df.drop(columns = [col], inplace = True)

df.drop(columns = ['color', 'name'], inplace = True)
# df.head(10)

In [62]:
gdf = gpd.GeoDataFrame(data = df, crs = "EPSG:4326", geometry = gpd.points_from_xy(df.longitude, df.latitude)) # type: ignore
gdf.drop(columns = ["longitude", "latitude"], inplace = True)
gdf = gdf.to_crs("EPSG:32737")

gdf['fid'] = gdf.index + 1
fid = gdf.pop('fid')
gdf.insert(0, 'fid', fid)
gdf.head(10)

,fid,altitude (m),desc,geometry
0,1,1521.2,Area,POINT (278867.357 9878331.137)
1,2,1522.5,Area,POINT (278973.918 9878345.402)
2,3,1522.7,Area,POINT (278968.116 9878465.611)
3,4,1523.2,Area,POINT (278965.180 9878531.578)
4,5,1522.2,Area,POINT (278872.928 9878533.664)
5,6,1520.8,Area,POINT (278818.728 9878524.602)
6,7,1520.1,Area,POINT (278752.434 9878500.666)
7,8,1521.2,Area,POINT (278867.357 9878331.137)
8,9,1521.6,Building,POINT (278870.516 9878469.212)
9,10,1522.8,Building,POINT (278937.880 9878471.998)


In [139]:
e = 1618010.606367111
f = -1741978.211608887

a = 0.9916765364496314
d = 1.175648448821448

b = -0.1635616585599564
c = 0.02551828620198648

In [140]:
# Transform to corresponding coordinates
gdf2 = gpd.GeoDataFrame((gdf.affine_transform([a, b, c, d, e, f])))
gdf2.head(10)


,0
0,POINT (278840.597 9878582.683)
1,POINT (278943.938 9878602.174)
2,POINT (278918.522 9878743.349)
3,POINT (278904.820 9878820.828)
4,POINT (278812.995 9878820.927)
5,POINT (278760.729 9878808.890)
6,POINT (278698.901 9878779.058)
7,POINT (278840.597 9878582.683)
8,POINT (278821.145 9878745.092)
9,POINT (278887.493 9878750.086)
